# TMDB에서 API 활용하여 평점기준으로 영화를 추천해주는 기능 구현
- ...을 통해서 requests, numpy, pandas 복습을 해보자

In [8]:
import requests
import pandas as pd
import numpy as np
from IPython.display import display

#API
api = '961b907ea3fefe1cbfe81d7f06b87762'

# url maker
def get_url(date):
    url = 'https://api.themoviedb.org/3/discover/movie'
    params = {
        'api_key': api,
        'language': 'ko-KR',
        'primary_release_date.lte': date,
        'sort_by':' vote_average.desc',
        'vote_count.gte': 50 # 최소 50개 이상 평점 투표된 영화만 필터링
    }
    return url, params

# TMDB API 요청 함수
def fetch_movies(url, params): 
    res = requests.get(url, params=params)
    
    if res.status_code == 200:
        return res.json().get("results", [])
    else:
        print('api 요청 실패:', res.status_code)
        return []

# 영화 데이터 가공 및 추천
def process_movies(movies):
    if not movies:
        print('해당 날짜에 개봉한 영화가 없습니다')
        return pd.DataFrame()

    df = pd.DataFrame(movies, columns=['title', 'release_date', 'vote_average', 'vote_count'])

    # 평점 정규화 (평점 / 10.0 으로 0~1 사이 값) - vote의 수가 많건 적건 모델의 판단을 공정하게 하기 위함
    df['normalized_rating'] = np.round(df['vote_average'] / 10, 2) # 소수점 둘째자리에서 반올림
    df = df.rename(columns={
        'title': '영화제목',
        'release_date': '개봉일',
        'vote_average': '평점',
        'normalized_rating': '정규화된 평점'
    })
    return df.sort_values(by='평점', ascending=False)

# 실행 코드
if __name__ == '__main__':
    date_input = input('영화 조회일(YYYY-MM-DD)을 입력하세요: ')
    api_url_tuple = get_url(date_input)
    url, params = api_url_tuple
    movies = fetch_movies(url, params)
    movie_df = process_movies(movies)

    if not movie_df.empty:
        print('🎞 추천 영화 목록:')
        display(movie_df[['영화제목', '개봉일', '평점', '정규화된 평점']].head(10))

영화 조회일(YYYY-MM-DD)을 입력하세요:  2023-01-01


🎞 추천 영화 목록:


,영화제목,개봉일,평점,정규화된 평점
13,쇼생크 탈출,1994-09-23,8.708,0.87
14,대부,1972-03-14,8.689,0.87
17,다크 나이트,2008-07-16,8.519,0.85
19,반지의 제왕: 왕의 귀환,2003-12-17,8.480,0.85
5,인터스텔라,2014-11-05,8.451,0.85
4,어벤져스: 인피니티 워,2018-04-25,8.238,0.82
12,탑건: 매버릭,2022-05-21,8.188,0.82
0,인디아나 존스: 레이더스,1981-06-12,7.921,0.79
16,타이타닉,1997-11-18,7.907,0.79
9,어벤져스,2012-04-25,7.729,0.77
